# create a git repo

```shell
> git init test
Initialized empty Git repository in /tmp/test/.git/
> cd test
> find .git/objects
.git/objects
.git/objects/info
.git/objects/pack
> find .git/objects -type f
>
# ie, there are no files currently in an empty repo
```

## Store an object

```bash
> echo 'test content' | git hash-object -w --stdin
d670460b4b4aece5915caf5c68d12f560a9fe3e4
```

* -w tells hash-object to store the object
  * otherwise, the command simply tells you what the key would be
* --stdin tells the command to read the content from stdin
  * if you don’t specify this, hash-object expects a file path at the end.
* The output from the command is a 40-character checksum hash.
* This is the SHA-1 hash – a checksum of the content you’re storing plus a header

```shell
> find .git/objects -type f
.git/objects/d6/70460b4b4aece5915caf5c68d12f560a9fe3e4
```

* You can see a file in the objects directory.
* This is how Git stores the content initially
  * as a single file per piece of content
  * named with the SHA-1 checksum of the content and its header
  * The subdirectory is named with the first 2 characters of the SHA-1
  * the filename is the remaining 38 characters.

### cat-file

* You can pull the content back out of Git with the cat-file command.
* This command is sort of a Swiss army knife for inspecting Git objects.
* Passing -p to it instructs the cat-file, ie, pretty printing

```shell
> git cat-file -p d670460b4b4aece5915caf5c68d12f560a9fe3e4
test content
```

## adding content

```shell
> echo 'version 1' &gt; test.txt
> git hash-object -w test.txt
83baae61804e65cc73a7201a7252750c76066a30
```

```shell
> echo 'version 2' &gt; test.txt
> git hash-object -w test.txt
1f7a7a472abf3dd9643fd615f6da379c4acb3e3a
```

```shell
> find .git/objects -type f
.git/objects/1f/7a7a472abf3dd9643fd615f6da379c4acb3e3a
.git/objects/83/baae61804e65cc73a7201a7252750c76066a30
.git/objects/d6/70460b4b4aece5915caf5c68d12f560a9fe3e4
```

```shell
> git cat-file -t 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a
blob
```

# Tree objects

```shell
> git cat-file -p master^{tree}
100644 blob a906cb2a4a904a152e80877d4088654daad0c859      README
100644 blob 8f94139338f9404f26296befa88755fc2598c289      Rakefile
040000 tree 99f1a6d12cb4b6f19c8655fca46c3ecf317074e0      lib
```

So, to create a tree object, you first have to set up an index by staging some files. To create an index with a single entry – the first version of your test.txt file – you can use the plumbing command update-index. You use this command to artificially add the earlier version of the test.txt file to a new staging area

* 83baae61804e65cc73a7201a7252750c76066a30 is the hash of the first version of test.txt
* --add : because the file doesn’t yet exist in your staging area 
* --cacheinfo : because the file you’re adding isn’t in your directory but is in your database
* 100644 : Mode
  * 100755, which means it’s an executable file
  * 120000, which specifies a symbolic link

```shell
git update-index --add --cacheinfo 100644 83baae61804e65cc73a7201a7252750c76066a30 test.txt
```

Now, you can use the write-tree command to write the staging area out to a tree object. No -w option is needed – calling write-tree automatically creates a tree object from the state of the index if that tree doesn’t yet exist:

```shell
> git write-tree
d8329fc1cc938780ffdd9f94e0d364e0ea74f579
> git cat-file -p d8329fc1cc938780ffdd9f94e0d364e0ea74f579
100644 blob 83baae61804e65cc73a7201a7252750c76066a30      test.txt
```

You can also verify that this is a tree object:
```shell
> git cat-file -t d8329fc1cc938780ffdd9f94e0d364e0ea74f579
tree
```

You’ll now create a new tree with the second version of test.txt and a new file as well:

```shell
-> echo 'new file' > new.txt
-> git update-index test.txt
-> git update-index --add new.txt
```

Your staging area now has the new version of test.txt as well as the new file new.txt. Write out that tree (recording the state of the staging area or index to a tree object) and see what it looks like:

```shell
-> git write-tree
0155eb4229851634a0f03eb265b69f5a2d56f341
-> git cat-file -p 0155eb4229851634a0f03eb265b69f5a2d56f341
100644 blob fa49b077972391ad58037050f2a75f74e3671e92      new.txt
100644 blob 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a      test.txt
```

Notice that this tree has both file entries and also that the test.txt SHA-1 is the “version 2” SHA-1 from earlier (1f7a7a). Just for fun, you’ll add the first tree as a subdirectory into this one. You can read trees into your staging area by calling read-tree. In this case, you can read an existing tree into your staging area as a subtree by using the --prefix option to read-tree:

```shell
-> git read-tree --prefix=bak d8329fc1cc938780ffdd9f94e0d364e0ea74f579
-> git write-tree
3c4e9cd789d88d8d89c1073707c3585e41b0e614
-> git cat-file -p 3c4e9cd789d88d8d89c1073707c3585e41b0e614
040000 tree d8329fc1cc938780ffdd9f94e0d364e0ea74f579      bak
100644 blob fa49b077972391ad58037050f2a75f74e3671e92      new.txt
100644 blob 1f7a7a472abf3dd9643fd615f6da379c4acb3e3a      test.txt
```

# Commit objects

You have three trees that specify the different snapshots of your project that you want to track, but the earlier problem remains: you must remember all three SHA-1 values in order to recall the snapshots. You also don’t have any information about who saved the snapshots, when they were saved, or why they were saved. This is the basic information that the commit object stores for you.

To create a commit object, you call commit-tree and specify a single tree SHA-1 and which commit objects, if any, directly preceded it. Start with the first tree you wrote:

```shell
-> echo 'first commit' | git commit-tree d8329f
fdf4fc3344e67ab068f836878b6c4951e3b15f3d
```

You will get a different hash value because of different creation time and author data. Replace commit and tag hashes with your own checksums further in this chapter. Now you can look at your new commit object with cat-file:

```shell
✗ git cat-file -p ae0b21e8f28f71fce63aa4123db21590a2fb9386
tree d8329fc1cc938780ffdd9f94e0d364e0ea74f579
author jvs <john.von.seneca@gmail.com> 1462024110 +0530
committer jvs <john.von.seneca@gmail.com> 1462024110 +0530

first commit
```

Next, you’ll write the other two commit objects, each referencing the commit that came directly before it:



```shell
✗ echo 'second commit' | git commit-tree 0155eb4229851634a0f03eb265b69f5a2d56f341 -p ae0b21e8f28f71fce63aa4123db21590a2fb9386
35a741bd9e435f5249fc8f9298319752b8e09e3c
✗ echo 'third commit'  | git commit-tree 3c4e9cd789d88d8d89c1073707c3585e41b0e614 -p 35a741bd9e435f5249fc8f9298319752b8e09e3c
46d99088da8b65c2d355eab938517022a096908c
```

Each of the three commit objects points to one of the three snapshot trees you created. Oddly enough, you have a real Git history now that you can view with the git log command, if you run it on the last commit SHA-1:

```shell
-> git log --stat 46d99088da8b65c2d355eab938517022a096908c
commit 46d99088da8b65c2d355eab938517022a096908c
Author: jvs <john.von.seneca@gmail.com>
Date:   Sat Apr 30 19:21:01 2016 +0530

    third commit

 bak/test.txt | 1 +
 1 file changed, 1 insertion(+)

commit 35a741bd9e435f5249fc8f9298319752b8e09e3c
Author: jvs <john.von.seneca@gmail.com>
Date:   Sat Apr 30 19:20:25 2016 +0530

    second commit

 new.txt  | 1 +
 test.txt | 2 +-
 2 files changed, 2 insertions(+), 1 deletion(-)

commit ae0b21e8f28f71fce63aa4123db21590a2fb9386
Author: jvs <john.von.seneca@gmail.com>
Date:   Sat Apr 30 19:18:30 2016 +0530

    first commit

 test.txt | 1 +
 1 file changed, 1 insertion(+)
 ```

```shell
✗ find .git/objects -type f
.git/objects/01/55eb4229851634a0f03eb265b69f5a2d56f341
.git/objects/1f/7a7a472abf3dd9643fd615f6da379c4acb3e3a
.git/objects/35/a741bd9e435f5249fc8f9298319752b8e09e3c
.git/objects/3c/4e9cd789d88d8d89c1073707c3585e41b0e614
.git/objects/46/d99088da8b65c2d355eab938517022a096908c
.git/objects/83/baae61804e65cc73a7201a7252750c76066a30
.git/objects/ae/0b21e8f28f71fce63aa4123db21590a2fb9386
.git/objects/bd/9dbf5aae1a3862dd1526723246b20206e5fc37
.git/objects/d8/329fc1cc938780ffdd9f94e0d364e0ea74f579
.git/objects/fa/49b077972391ad58037050f2a75f74e3671e92
```

# Object storage

```shell
-> irb
```

```ruby
irb(main):001:0> content = 'what is up, doc?'
=> "what is up, doc?"
irb(main):002:0> head = "blob #{content.length}"
=> "blob 16"
irb(main):003:0> head = "blob #{content.length}\0"
=> "blob 16\u0000"
irb(main):005:0> store = head + content
=> "blob 16\u0000what is up, doc?"
```

Git constructs a header that starts with the type of the object, in this case a blob. Then, it adds a space followed by the size of the content and finally a null byte:

Git concatenates the header and the original content and then calculates the SHA-1 checksum of that new content

You can calculate the SHA-1 value of a string in Ruby by including the SHA1 digest library with the require command and then calling Digest::SHA1.hexdigest() with the string

```ruby
irb(main):006:0> require 'digest/sha1'
=> true
irb(main):007:0> sha1 = Digest::SHA1.hexdigest(store)
=> "bd9dbf5aae1a3862dd1526723246b20206e5fc37"
```


Git compresses the new content with zlib, which you can do in Ruby with the zlib library. First, you need to require the library and then run Zlib::Deflate.deflate() on the content:

```ruby
irb(main):008:0> require 'zlib'
=> true
irb(main):009:0> zlib_content = Zlib::Deflate.deflate(store)
=> "x\x9CK\xCA\xC9OR04c(\xCFH,Q\xC8,V(-\xD0QH\xC9O\xB6\a\x00_\x1C\a\x9D"
irb(main):010:0> path = ".git/objects/#{sha1[0,2]}/#{sha1[2,38]}"
=> ".git/objects/bd/9dbf5aae1a3862dd1526723246b20206e5fc37"
```

Finally, you’ll write your zlib-deflated content to an object on disk. You’ll determine the path of the object you want to write out (the first two characters of the SHA-1 value being the subdirectory name, and the last 38 characters being the filename within that directory). In Ruby, you can use the FileUtils.mkdir_p() function to create the subdirectory if it doesn’t exist. Then, open the file with File.open() and write out the previously zlib-compressed content to the file with a write() call on the resulting file handle:

```ruby
irb(main):011:0> require 'fileutils'
=> true
irb(main):012:0> FileUtils.mkdir_p(File.dirname(path))
=> [".git/objects/bd"]
irb(main):013:0> File.open(path, 'w') {|f| f.write(zlib_content)}
=> 32
irb(main):014:0>
```

```shell
```